![Python](https://img.shields.io/badge/python-3.9-blue)
![Status: Pending Migration](https://img.shields.io/badge/status-pending%20migration-orange)

<a id="table-of-contents"></a>
# 📖 Deep Learning

- [🧠 Why Go Deeper?](#why-go-deeper)
- [🔧 Setup Problem and Dataset](#setup-problem-and-dataset)
- [🏗️ Build Deep MLP](#build-deep-mlp)
    - [🧱 Define Architecture](#define-architecture)
    - [🚀 Train Deep MLP](#train-deep-mlp)
    - [📈 Visualize Loss and Decision Boundary](#visualize-deep-mlp-results)
- [🧠 High-Level Intro to CNNs](#intro-to-cnns)
    - [🔲 When Do You Need CNNs?](#cnn-use-cases)
    - [⚙️ CNN Layers Explained](#cnn-building-blocks)
    - [📦 CNN with Keras/Torch](#cnn-code-demo)
- [🔚 Closing Notes](#closing-notes)
___

<a id="why-go-deeper"></a>
# 🧠 Why Go Deeper?



[Back to the top](#table-of-contents)
___


<a id="setup-problem-and-dataset"></a>
# 🔧 Setup Problem and Dataset



[Back to the top](#table-of-contents)
___


<a id="build-deep-mlp"></a>
# 🏗️ Build Deep MLP



<a id="define-architecture"></a>
#### 🧱 Define Architecture



<a id="train-deep-mlp"></a>
#### 🚀 Train Deep MLP



<a id="visualize-deep-mlp-results"></a>
#### 📈 Visualize Loss and Decision Boundary



[Back to the top](#table-of-contents)
___


<a id="intro-to-cnns"></a>
# 🧠 High-Level Intro to CNNs



<a id="cnn-use-cases"></a>
#### 🔲 When Do You Need CNNs?



<a id="cnn-building-blocks"></a>
#### ⚙️ CNN Layers Explained



<a id="cnn-code-demo"></a>
#### 📦 CNN with Keras/Torch



[Back to the top](#table-of-contents)
___


<a id="closing-notes"></a>
# 🔚 Closing Notes


[Back to the top](#table-of-contents)
___
